In [ ]:
""" Colab gdrive utils """
from google.colab import drive
import os

!ln -s "/content/drive/My Drive/Colab Notebooks/ML3D" "/content"

def mount_drive():
    drive.mount('/content/drive')
    print(os.listdir('.'))
    os.chdir('/content/ML3D')
    print(f'Working directory set to: \"{os.getcwd()}\"')

def unmount_drive():
    drive.flush_and_unmount()
    os.chdir('/content')

unmount_drive()
mount_drive()

In [ ]:
!pip install tensorboardX
!pip install trimesh
!pip install in_place

In [ ]:
import trimesh
import glob
import numpy as np
import os 

path = os.getcwd()
path = os.path.join(*[path, 'data', 'mixamo', 'objfiles'])
objfiles = glob.glob(path + "/*.obj")
objfiles

In [ ]:
def parse_scene(scene):
    try:

        keys = list(scene.geometry.keys())
        g_verts = []
        g_colors =[]
        for k in keys:
            print(k)
            colors = scene.geometry[k].visual.to_color().vertex_colors
            verts = np.asarray(scene.geometry[k].vertices)
            g_colors.append(colors)
            g_verts.append(verts)
        print("This is a scene, there are multiple meshes inside, and they are concatenated")
    except:
        g_verts = []
        g_colors =[]
        colors = scene.visual.to_color().vertex_colors
        verts = np.asarray(scene.vertices)
        g_colors.append(colors)
        g_verts.append(verts)
        print("This is a single mesh")
    return np.concatenate(g_verts),np.concatenate(g_colors)[:,:3]

In [ ]:
meshes = []
for idx, o in enumerate(objfiles):
    if idx < 135:
        continue
    f = open(o,"r") 
    scene = trimesh.load_mesh(f,"obj")
    verts,colors = parse_scene(scene)
    np.save(o[:o.find(".obj")] + ".npy",np.stack([verts,colors],axis=1))
    print(idx, len(objfiles))
    #print("Printing verts and colors as sanity check ",verts,colors)

npys = glob.glob(path + "/*.npy")
print(f'Number of converted files:{len(npys)}')